In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, concatenate, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from net_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.callbacks import EarlyStopping
# %matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Sequential

import PIL

Using TensorFlow backend.


In [2]:
dataSrc = "data/"
imageSrc = "data/ImageData/"

trainCSV = dataSrc + "train_image.csv"
testCSV = dataSrc + "test_image.csv"

In [22]:
def CTRModel(csr_input_shape = (7,1), weights_path=None):    #建立CTR模型
    
    ConvModel = VGG19(weights='imagenet')
    Image_INPUT = ConvModel.input
    ConvModel = Model(inputs=Image_INPUT, outputs=ConvModel.get_layer('flatten').output)
    
    for layer in ConvModel.layers:
        layer.trainable = False
    ConvModel.layers[-3].trainable = True
    ConvModel.layers[-4].trainable = True
    Convnet = ConvModel.output
#     Convnet = Dense(128, name='Convnet_fc1')(Convnet)
#     Convnet = Activation('relu')(Convnet)
    
    CSR_INPUT = Input(csr_input_shape)
    Basenet = Flatten()(CSR_INPUT)
    Basenet = Dense(128, name='Basenet_fc1')(Basenet)
    Basenet = Activation('relu')(Basenet)
    
    CTRNet = concatenate([Basenet, Convnet])

    CTRNet = BatchNormalization(name = "Bn_1")(CTRNet)
    
    CTRNet = Dense(256, activation='sigmoid', name='Combnet_fc1')(CTRNet)

    CTRNet = Dense(1, activation='sigmoid', name='Combnet_fc2')(CTRNet)
    
    if weights_path:  
        CTRNet.load_weights(weights_path)
        
    CTRModel = Model(inputs = [Image_INPUT, CSR_INPUT], outputs = CTRNet, name='CTRModel')
    
    return CTRModel  

In [4]:
X_train_img_orig, X_train_csr, Y_train_orig, X_test_img_orig, X_test_csr, Y_test_orig, = load_dataset_onehot()

In [5]:
print ("number of training examples = " + str(X_train_img_orig.shape[0]))
print ("number of test examples = " + str(X_test_img_orig.shape[0]))
print ("Img_train shape: " + str(X_train_img_orig.shape))
print ("CSR_train shape: " + str(X_train_csr.shape))
print ("Y_train shape: " + str(Y_train_orig.transpose().shape))

print ("Img_test shape: " + str(X_test_img_orig.shape))
print ("CSR_test shape: " + str(X_test_csr.shape))
print ("Y_test shape: " + str(Y_test_orig.transpose().shape))

number of training examples = 5000
number of test examples = 1000
Img_train shape: (5000, 224, 224, 3)
CSR_train shape: (5000, 7, 1)
Y_train shape: (5000, 1)
Img_test shape: (1000, 224, 224, 3)
CSR_test shape: (1000, 7, 1)
Y_test shape: (1000, 1)


In [23]:
ctr = CTRModel( csr_input_shape = [X_train_csr.shape[1],X_train_csr.shape[2]] )
ctr.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
# E = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')
ctr.fit([X_train_img_orig, X_train_csr], Y_train_orig.transpose(), epochs = 5, batch_size = 32, validation_split = 0.05)

In [ ]:
# img_path = imageSrc + 'True/01e5c09cgy1flam1h04p8j215j1me7gt.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# block5_pool_features = model.predict(x)

In [ ]:
ctr.save("model.h5")